In [1]:
from vmtk import pypes
from vmtk import vmtkscripts

In [2]:
# String containing cases path and filenames
parentDir = "/home/iagolessa/documents/unesp/doctorate/aneurysms/geometries/vrCases/"

# Cases storage directory
casesDir = "originalCases/"

# Saving location
vmtkReconstructionDir = "vmtkReconstruction/"

# Storage location and file names
caseNameDir = "71_MLS/"
caseId = "case71"

casePath = parentDir+vmtkReconstructionDir+caseNameDir
parentVesselDir = parentDir+vmtkReconstructionDir+caseNameDir+"parentVessel/"

# The store magic command, to access a variable across notebooks
# %store casePath

# This has deleted the variable
# del casePath

# Extensions
imageExtension = ".vti"
surfaceExtension = ".vtp"

In [3]:
# Surface reader
def readSurface(surfaceFileName):
    """ Function to read a surface VTK object from file name.
        The function returns a surface VTK object. """
    surfaceReader = vmtkscripts.vmtkSurfaceReader()
    surfaceReader.InputFileName = surfaceFileName
    surfaceReader.Execute()
    
    return surfaceReader.Surface

# Viewing surface
def viewSurface(surface):
    """Function for visualize VTK surface objects"""
    surfaceViewer = vmtkscripts.vmtkSurfaceViewer()
    surfaceViewer.Surface = surface
    surfaceViewer.Execute()

<hr>
# MAPPING AND PATCHING

This tutorial demonstrates how to map the surface of a population of vessels onto the same parametric space and enable statistical analyses of surface-based quantities.

## Longitudinal and circumferential metrics
By means of the vmtkbranchmetrics script two additional arrays are created on each branch of the split surface whose default names are AbscissaMetric and AngularMetric: the first is computed from the curvilinear abscissa defined on the centerlines, while the second represents the periodic circumferential coordinate of mesh points around the centerlines and spans the interval (-Ï, +Ï). In Figure 3 iso-contours over the two arrays are also shown.

In [5]:
branchMetrics = vmtkscripts.vmtkBranchMetrics()

branchMetrics.SurfaceInputFileName = casePath+'surfaces/branchClipped.vtp'
branchMetrics.CenterlinesInputFileName = casePath+'centerlines/branchExtracted.vtp'
branchMetrics.IORead()

branchMetrics.AbscissasArrayName = 'Abscissas'
branchMetrics.NormalsArrayName = 'ParallelTransportNormals'
branchMetrics.CenterlineIdsArrayName = 'CenterlineIds'
branchMetrics.GroupIdsArrayName = 'GroupIds'
branchMetrics.TractIdsArrayName = 'TractIds'
branchMetrics.BlankingArrayName = 'Blanking'
branchMetrics.RadiusArrayName = 'MaximumInscribedSphereRadius'

branchMetrics.SurfaceOutputFileName = casePath+'branchAnalysis/surfaceClippedMetrics.vtp'

branchMetrics.PrintInputMembers()
branchMetrics.PrintOutputMembers()

branchMetrics.Execute()
branchMetrics.IOWrite()

Reading VTK XML surface file.

Reading VTK XML surface file.

Input vmtkbranchmetrics members:

    Id = 0

    Disabled = 0

    Surface = vtkPolyData

    SurfaceInputFileName = /home/iagolessa/documents/unesp/doctorate/aneurysms/geometries/vrCases/vmtkReconstruction/71_MLS/surfaces/branchClipped.vtp

    Centerlines = vtkPolyData

    CenterlinesInputFileName = /home/iagolessa/documents/unesp/doctorate/aneurysms/geometries/vrCases/vmtkReconstruction/71_MLS/centerlines/branchExtracted.vtp

    ComputeAbscissaMetric = 1

    ComputeAngularMetric = 1

    AbscissasArrayName = Abscissas

    NormalsArrayName = ParallelTransportNormals

    GroupIdsArrayName = GroupIds

    CenterlineIdsArrayName = CenterlineIds

    TractIdsArrayName = TractIds

    RadiusArrayName = MaximumInscribedSphereRadius

    BlankingArrayName = Blanking

    AngularMetricArrayName = AngularMetric

    AbscissaMetricArrayName = AbscissaMetric

    SurfaceOutputFileName = /home/iagolessa/documents/unesp/doctorate

# Metrics mapping to branches
By construction of a harmonic function (Figure 4-left) over each vascular segment, vmtkbranchmapping maps and stretches the longitudinal metric to correctly account for the presence of insertion regions at bifurcations; the additional StretchedMapping array is added to the surface (Figure 4-middle).

In [6]:
branchMapping = vmtkscripts.vmtkBranchMapping()

# Read surface or use the calculated above
# branchMapping.SurfaceInputFileName = casePath+'surfaces/branchClipped.vtp'
branchMapping.Surface = branchMetrics.Surface
branchMapping.CenterlinesInputFileName = casePath+'centerlines/branchExtracted.vtp'
branchMapping.ReferenceSystemsInputFileName = casePath+'centerlines/bifurcationRefSystem.vtp'
branchMapping.IORead()

branchMapping.AbscissasArrayName = 'Abscissas'
branchMapping.NormalsArrayName = 'ParallelTransportNormals'
branchMapping.CenterlineIdsArrayName = 'CenterlineIds'
branchMapping.GroupIdsArrayName = 'GroupIds'
branchMapping.TractIdsArrayName = 'TractIds'
branchMapping.BlankingArrayName = 'Blanking'
branchMapping.RadiusArrayName = 'MaximumInscribedSphereRadius'

branchMapping.AngularMetricArrayName = 'AngularMetric' # from cell above (branch metrics)
branchMapping.AbscissaMetricArrayName = 'AbscissaMetric' # from cell above (branch metrics)

branchMapping.SurfaceOutputFileName = casePath+'branchAnalysis/surfaceClippedMapping.vtp'

branchMapping.PrintInputMembers()
branchMapping.PrintOutputMembers()

branchMapping.Execute()
branchMapping.IOWrite()

Reading VTK XML surface file.

Reading VTK XML surface file.

Input vmtkbranchmapping members:

    Id = 0

    Disabled = 0

    Surface = vtkPolyData

    SurfaceInputFileName = 

    Centerlines = vtkPolyData

    CenterlinesInputFileName = /home/iagolessa/documents/unesp/doctorate/aneurysms/geometries/vrCases/vmtkReconstruction/71_MLS/centerlines/branchExtracted.vtp

    ReferenceSystems = vtkPolyData

    ReferenceSystemsInputFileName = /home/iagolessa/documents/unesp/doctorate/aneurysms/geometries/vrCases/vmtkReconstruction/71_MLS/centerlines/bifurcationRefSystem.vtp

    AbscissasArrayName = Abscissas

    NormalsArrayName = ParallelTransportNormals

    GroupIdsArrayName = GroupIds

    CenterlineIdsArrayName = CenterlineIds

    TractIdsArrayName = TractIds

    ReferenceSystemsNormalArrayName = 

    RadiusArrayName = MaximumInscribedSphereRadius

    BlankingArrayName = Blanking

    AngularMetricArrayName = AngularMetric

    HarmonicMappingArrayName = HarmonicMapping

    

In [12]:
branchGeometry = vmtkscripts.vmtkBranchGeometry()

branchGeometry.PrintInputMembers()
branchGeometry.PrintOutputMembers()

Input vmtkbranchgeometry members:

    Id = 0

    Disabled = 0

    Centerlines = None

    CenterlinesInputFileName = 

    RadiusArrayName = 

    GroupIdsArrayName = 

    BlankingArrayName = 

    LengthArrayName = Length

    CurvatureArrayName = Curvature

    TorsionArrayName = Torsion

    TortuosityArrayName = Tortuosity

    LineSmoothing = 0

    NumberOfSmoothingIterations = 100

    SmoothingFactor = 0.1

    GeometryDataOutputFileName = 

Output vmtkbranchgeometry members:

    Id = 0

    GeometryData = None

    LengthArrayName = Length

    CurvatureArrayName = Curvature

    TorsionArrayName = Torsion

    TortuosityArrayName = Tortuosity


<hr>
# Patching of surface mesh and attributes
All the ingredients are now in place to perform the real patching of the surface, that is to âcutâ a set of contiguous rectangular regions on the mesh that follow iso-contours in the StretchedMapping and AngularMetric arrays; all the quantities of interest (WSS and OSI in this case) are averaged over these areas.

By means of the options -longitudinalpatchsize and -circularpatches we impose the dimensions of the patches, in terms of âheightâ (in mm) of the patch along the longitudinal direction and number of angular cut over the interval (-Ï, +Ï) respectively; the result of this discretization can be seen visualizing the Slab and Sector arrays created by the script or the mesh new surface discretization (Figure 5). Youâll probably have to play with these values to find out the discretization that best fits your needs; we here cut every 0.5 mm and subdivide the interval (-Ï, +Ï) in 12 sectors.

In [ ]:
vmtkbranchpatching -ifile aorta_clipped_mapping.vtp 
    -groupidsarray GroupIds 
    -longitudinalmappingarray StretchedMapping 
    -circularmappingarray AngularMetric 
    -longitudinalpatchsize 0.5 
    -circularpatches 12 
    -ofile aorta_clipped_patching.vtp

In [17]:
branchPatching = vmtkscripts.vmtkBranchPatching()

branchPatching.SurfaceInputFileName = casePath+'branchAnalysis/surfaceClippedMapping.vtp'
branchPatching.IORead()
branchPatching.GroupIdsArrayName = 'GroupIds'
branchPatching.LongitudinalMappingArrayName = 'StretchedMapping'
branchPatching.CircularMappingArrayName = 'AngularMetric'
branchPatching.LongitudinalPatchSize = 0.5
branchPatching.CircularNumberOfPatches = 12

branchPatching.SurfaceOutputFileName = casePath+'branchAnalysis/surfaceClippedPatching.vtp'

branchPatching.PrintInputMembers()
branchPatching.PrintOutputMembers()

branchPatching.Execute()
branchPatching.IOWrite()

Reading VTK XML surface file.

Input vmtkbranchpatching members:

    Id = 0

    Disabled = 0

    Surface = vtkPolyData

    SurfaceInputFileName = /home/iagolessa/documents/unesp/doctorate/aneurysms/geometries/vrCases/vmtkReconstruction/38_RSS/branchAnalysis/surfaceClippedMapping.vtp

    PatchSize = [0.0, 0.0]

    LongitudinalPatchSize = 0.5

    CircularNumberOfPatches = 12

    CircularPatching = 1

    UseConnectivity = 1

    GroupIdsArrayName = GroupIds

    LongitudinalMappingArrayName = StretchedMapping

    CircularMappingArrayName = AngularMetric

    LongitudinalPatchNumberArrayName = Slab

    CircularPatchNumberArrayName = Sector

    PatchAreaArrayName = PatchArea

    SurfaceOutputFileName = /home/iagolessa/documents/unesp/doctorate/aneurysms/geometries/vrCases/vmtkReconstruction/38_RSS/branchAnalysis/surfaceClippedPatching.vtp

    PatchedDataOutputFileName = 

Output vmtkbranchpatching members:

    Id = 0

    Surface = vtkPolyData

    PatchedData = None

    Pat